<h1>Task 3: Create a more sophisticated algorithm for embedding hidden messages </h1>

Import libaries

In [1]:
import wave
import array

Creating the function for embedding the message into the sound file

In [2]:
def kthLeastSignificantBit(soundfile, newfile, msg, k):
    #ensure k is less than 8 so that most significant bit is not changed and greater than 0
    if k > 0 and k < 8:
        #reading the file, getting sound paramters and converting it into a byte array
        sound = wave.open(soundfile, mode="rb")
        sound_params = sound.getparams()
        byteArray = bytearray(list(sound.readframes(sound.getnframes())))
        sound.close()
        
        #find the difference in length between the end of the msg to the end of the song
        padding = int((len(byteArray) - len(msg) * 8 * 8) / 8)
        
        #pad the message with #
        msg = msg + padding * '#'
        
        #convert the text to an array
        #First we iterate through the message to get the characters
        #We will then convert the characters to unicode and then converting the unicode to binary
        #We will then strip 0b which is the prefix we get after converting it to binary and right align the string
        #Finally we will use the map function to convert all of them to integers
        bitsArray = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8,'0') for i in msg])))
            
        #For each bit in the bit array, we take the same byte in the array and set it all to 0 until the nth bit from the right
        #Next, we will do a left shift of the bit by k places so that it will be located at the k position
        #The OR operator will then set the kth least significant bit to the bit value
        for i, bit in enumerate(bitsArray):
            byteArray[i] = (byteArray[i] & 255 - 2**k) | bit << k
            
        #get the final byte array and ensure that it cannot be modified
        byte_array_write = bytes(byteArray)
        
        #write the new sound with the embedded messge in
        with wave.open(newfile, 'wb') as writefile:
            writefile.setparams(sound_params)
            writefile.writeframes(byte_array_write)
            
    else:
        print("k should be between 0 and 8")

Creating the function that would extract the message from the sound file

In [3]:
def extractMessage(soundfile, k):
    #ensure that k is less than 8 and greater than 0
    if k > 0 and k < 8:
        
        #read the file and create the byte array
        sound = wave.open(soundfile, mode="rb")
        byteArray = array.array("B")
        byteArray.fromstring(sound.readframes(sound.getnframes()))
        sound.close()
        
        extracted = []
        
        #iterate through the byteArray finding the nth bit which should be set to 1
        #we would then do a right shift of the bit by k places so that we can extract the value
        for i in range(len(byteArray)):
            extracted.append((byteArray[i] & 2 ** k) >> k)
        
        #next we change the byte array back to a string
        #we loop through the extracted array in intervals of 8bits
        #we then use the map function to iterate through the entire byte and convert it into a string
        #next, we iterate through the map object and concatenate it into an 8bit string
        #finally we convert the 8bit string into an integer and then concatenate it into a string
        finalString = ""
        for i in range (0, len(extracted), 8):
            firstString = map(str, extracted[i : i + 8])
            secondString = ""
            for char in firstString:
                secondString += char
            thirdString = (int(secondString, 2))
            finalString += chr(thirdString)
        
        #split the string using # and getting the first element to get the secret message
        hidden_message = finalString.split("#")[0]
        print(hidden_message)
        
    else:
        print("k should be between 0 and 8")

Testing the encoding of the message and let k be 4

In [4]:
soundfile = "sounds/Ex3_sound5.wav"
newfile = "sounds/Ex3_sound5_embedded.wav"
message = "Father Christmas does not exist"

kthLeastSignificantBit(soundfile, newfile, message, 4)

Now testing the extracting of the message where k = 4

In [5]:
extractMessage(newfile, 4)

<ipython-input-5-f1d015504127>:1: DeprecationWarning: fromstring() is deprecated. Use frombytes() instead.
  extractMessage(newfile, 4)


Father Christmas does not exist
